<a href="https://colab.research.google.com/github/msakthiganesh/cse-576-nlp/blob/main/2_Implementation_Word_Importance_with_Attention_Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Dec  4 03:38:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd drive/MyDrive/NLP/

/content/drive/MyDrive/NLP


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import json
import glob
import pandas as pd
from tqdm.notebook import tqdm
import pickle
from transformers import RobertaModel, RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig, get_linear_schedule_with_warmup, AdamW
import time, datetime
import torch
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

In [5]:
# df = pd.read_csv("data/gpt3_data.csv", index_col=None)

In [6]:
# test_df = pd.read_csv("data/test_data.csv", index_col = None)

In [7]:
with open ('/content/drive/MyDrive/NLP/data/train_premise', 'rb') as fp:
    train_premise = pickle.load(fp)
with open ('/content/drive/MyDrive/NLP/data/train_hypothesis', 'rb') as fp:
    train_hypothesis = pickle.load(fp)
with open ('/content/drive/MyDrive/NLP/data/train_labels', 'rb') as fp:
    train_labels = pickle.load(fp)

with open ('/content/drive/MyDrive/NLP/data/val_premise', 'rb') as fp:
    val_premise = pickle.load(fp)
with open ('/content/drive/MyDrive/NLP/data/val_hypothesis', 'rb') as fp:
    val_hypothesis = pickle.load(fp)
with open ('/content/drive/MyDrive/NLP/data/val_labels', 'rb') as fp:
    val_labels = pickle.load(fp)

with open ('/content/drive/MyDrive/NLP/data/test_premise', 'rb') as fp:
    test_premise = pickle.load(fp)
with open ('/content/drive/MyDrive/NLP/data/test_hypothesis', 'rb') as fp:
    test_hypothesis = pickle.load(fp)
with open ('/content/drive/MyDrive/NLP/data/test_labels', 'rb') as fp:
    test_labels = pickle.load(fp)

In [8]:
train_premise[0], train_hypothesis[0], train_labels[0]

('If I am a good student, I will get a good grade.. If I am a bad student, I will get a bad grade.. Either I am a good student or a bad student.',
 'Therefore I will get a good grade.',
 0)

In [9]:
with open ('/content/drive/MyDrive/NLP/train_explain', 'rb') as fp:
    train_explain = pickle.load(fp)

with open ('/content/drive/MyDrive/NLP/val_explain', 'rb') as fp:
    val_explain = pickle.load(fp)

train_exp = []
for item in train_explain:
  train_exp.append(dict(item))


val_exp = []
for item in val_explain:
  val_exp.append(dict(item))

In [10]:
# MODEL_PATH = "best_rob_model/"
MODEL_PATH = "roberta-base"

In [11]:
config = RobertaConfig.from_pretrained(MODEL_PATH, num_labels=3) # hidden_size=768 must be a multiple of num_attention_heads
model = RobertaForSequenceClassification.from_pretrained(MODEL_PATH, 
                                                         config=config
                                                         )
roberta_tok = RobertaTokenizer.from_pretrained(MODEL_PATH)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [12]:
max_len=0
for i in tqdm(range(0,len(train_premise))):
  input_ids = roberta_tok.encode(train_premise[i], train_hypothesis[i], add_special_tokens=True)
  max_len = max(max_len, len(input_ids))

print(f"Max Length in Train Dataset: {max_len}")

  0%|          | 0/1633 [00:00<?, ?it/s]

Max Length in Train Dataset: 156


In [13]:
def encode_input(premise, hypothesis, label, maxlen):

  encoded_input_ids = []
  encoded_input_attn_mask = []
  exp_mask = []

  for i in tqdm(range(0, len(premise))):
    encoded_input = roberta_tok.encode_plus(premise[i],
                                        hypothesis[i],
                                        add_special_tokens=True,
                                        max_length=maxlen,
                                        return_attention_mask=True,
                                        return_tensors="pt",
                                        truncation=True,
                                        padding="max_length")
    
    encoded_input_ids.append(encoded_input.input_ids)
    encoded_input_attn_mask.append(encoded_input.attention_mask)

  encoded_input_ids = torch.cat(encoded_input_ids, dim=0)
  encoded_input_attn_mask = torch.cat(encoded_input_attn_mask, dim=0)
  label = torch.tensor(label, dtype=torch.long)

  return (encoded_input_ids, encoded_input_attn_mask, label)

In [14]:
train_encoded_input_ids, train_encoded_input_attn_mask, train_label = encode_input(premise = train_premise, hypothesis = train_hypothesis, label = train_labels, maxlen = 256)
val_encoded_input_ids, val_encoded_input_attn_mask, val_label = encode_input(premise = val_premise, hypothesis = val_hypothesis, label = val_labels, maxlen = 256)
test_encoded_input_ids, test_encoded_input_attn_mask, test_label = encode_input(premise = test_premise, hypothesis = test_hypothesis, label = test_labels, maxlen = 256)

  0%|          | 0/1633 [00:00<?, ?it/s]

  0%|          | 0/409 [00:00<?, ?it/s]

  0%|          | 0/1205 [00:00<?, ?it/s]

In [15]:
print(f"Original Premise 1: {train_premise[0]}")
print(f"Original Hypothesis 2: {train_hypothesis[0]}")
print(f"Token IDs: {train_encoded_input_ids[0]}")
print(f"Attention Mask: {train_encoded_input_attn_mask[0]}")
print(f"Label: {train_labels[0]}")

Original Premise 1: If I am a good student, I will get a good grade.. If I am a bad student, I will get a bad grade.. Either I am a good student or a bad student.
Original Hypothesis 2: Therefore I will get a good grade.
Token IDs: tensor([    0,  1106,    38,   524,    10,   205,  1294,     6,    38,    40,
          120,    10,   205,  4978,  7586,   318,    38,   524,    10,  1099,
         1294,     6,    38,    40,   120,    10,  1099,  4978,  7586, 15482,
           38,   524,    10,   205,  1294,    50,    10,  1099,  1294,     4,
            2,     2, 32730,    38,    40,   120,    10,   205,  4978,     4,
            2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,   

In [16]:
train_encoded_input_attn_mask[0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [18]:
train_exp_mask, train_bos, train_eos, train_pad, train_expnf = [], [], [], [], []

for i, tids in enumerate(train_encoded_input_ids):
  tid = tids.tolist()

  _decoded = []
  for _x in tid:
    _decoded.append(roberta_tok.decode([_x]))
  
  nm = []
  k = list(train_exp[i].keys())
  bos, eos, pad, expnf= [], [], [], []
  for idx, val in enumerate(_decoded):
    val = val.lstrip()
    if val == "<s>":
      nm.append(np.nan)
      bos.append(idx)
    elif val == "</s>":
      nm.append(np.nan)
      eos.append(idx)
    elif val == "<pad>":
      nm.append(np.nan)
      pad.append(idx)
    else:
      if val in k:
        nm.append(train_exp[i][val])
      else:
        nm.append(np.nan)
        expnf.append(idx)

  train_exp_mask.append(nm)
  train_bos.append(bos)
  train_eos.append(eos)
  train_pad.append(pad)
  train_expnf.append(expnf)



train_norm_exp_mask = []
for exp_mask in train_exp_mask:
  x = np.array(exp_mask).reshape(-1, 1)
  _fit_mask = scaler.fit_transform(x).flatten()
  train_norm_exp_mask.append(_fit_mask)

In [19]:
val_exp_mask, val_bos, val_eos, val_pad, val_expnf = [], [], [], [], []

for i, tids in enumerate(val_encoded_input_ids):
  tid = tids.tolist()

  _decoded = []
  for _x in tid:
    _decoded.append(roberta_tok.decode([_x]))
  
  nm = []
  k = list(val_exp[i].keys())
  bos, eos, pad, expnf= [], [], [], []
  for idx, val in enumerate(_decoded):
    val = val.lstrip()
    if val == "<s>":
      nm.append(np.nan)
      bos.append(idx)
    elif val == "</s>":
      nm.append(np.nan)
      eos.append(idx)
    elif val == "<pad>":
      nm.append(np.nan)
      pad.append(idx)
    else:
      if val in k:
        nm.append(val_exp[i][val])
      else:
        nm.append(np.nan)
        expnf.append(idx)

  val_exp_mask.append(nm)
  val_bos.append(bos)
  val_eos.append(eos)
  val_pad.append(pad)
  val_expnf.append(expnf)



val_norm_exp_mask = []
for exp_mask in val_exp_mask:
  x = np.array(exp_mask).reshape(-1, 1)
  _fit_mask = scaler.fit_transform(x).flatten()
  val_norm_exp_mask.append(_fit_mask)

In [20]:
len(train_norm_exp_mask), len(val_norm_exp_mask)

(1633, 409)

In [21]:
train_norm_exp_mask[0]

array([       nan, 0.52252351, 0.19205662, 0.57701393,        nan,
       0.51251089, 0.47107479,        nan, 0.19205662, 0.69509159,
       0.84183087,        nan, 0.51251089, 1.        ,        nan,
       0.52252351, 0.19205662, 0.57701393,        nan, 0.46894418,
       0.47107479,        nan, 0.19205662, 0.69509159, 0.84183087,
              nan, 0.46894418, 1.        ,        nan, 0.        ,
       0.19205662, 0.57701393,        nan, 0.51251089, 0.47107479,
       0.67784909,        nan, 0.46894418, 0.47107479,        nan,
              nan,        nan, 0.57910706, 0.19205662, 0.69509159,
       0.84183087,        nan, 0.51251089, 1.        ,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        

In [22]:
for i, nmask in enumerate(train_norm_exp_mask):

  train_bos_vals = train_bos[i]
  train_eos_vals = train_eos[i]
  train_pad_vals = train_pad[i]
  train_expnf_vals = train_expnf[i]

  for bos_val in train_bos_vals:
    nmask[bos_val] = 1 
  
  for bos_val in train_eos_vals:
    nmask[bos_val] = 1

  for bos_val in train_pad_vals:
    nmask[bos_val] = 0
  
  for bos_val in train_expnf_vals:
    nmask[bos_val] = 1

train_encoded_input_attn_mask = torch.from_numpy(np.array(train_norm_exp_mask))


for i, nmask in enumerate(val_norm_exp_mask):

  val_bos_vals = val_bos[i]
  val_eos_vals = val_eos[i]
  val_pad_vals = val_pad[i]
  val_expnf_vals = val_expnf[i]


  for bos_val in val_bos_vals:
    nmask[bos_val] = 1 
  
  for bos_val in val_eos_vals:
    nmask[bos_val] = 1

  for bos_val in val_pad_vals:
    nmask[bos_val] = 0
  
  for bos_val in val_expnf_vals:
    nmask[bos_val] = 1

val_encoded_input_attn_mask = torch.from_numpy(np.array(val_norm_exp_mask))

In [23]:
train_ds = TensorDataset(train_encoded_input_ids, train_encoded_input_attn_mask, train_label)
eval_ds = TensorDataset(val_encoded_input_ids, val_encoded_input_attn_mask, val_label)
test_ds= TensorDataset(test_encoded_input_ids, test_encoded_input_attn_mask, test_label)

In [24]:
train_encoded_input_attn_mask[0]

tensor([1.0000, 0.5225, 0.1921, 0.5770, 1.0000, 0.5125, 0.4711, 1.0000, 0.1921,
        0.6951, 0.8418, 1.0000, 0.5125, 1.0000, 1.0000, 0.5225, 0.1921, 0.5770,
        1.0000, 0.4689, 0.4711, 1.0000, 0.1921, 0.6951, 0.8418, 1.0000, 0.4689,
        1.0000, 1.0000, 0.0000, 0.1921, 0.5770, 1.0000, 0.5125, 0.4711, 0.6778,
        1.0000, 0.4689, 0.4711, 1.0000, 1.0000, 1.0000, 0.5791, 0.1921, 0.6951,
        0.8418, 1.0000, 0.5125, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [25]:
config = {
    "eos_token" : "[SEP]",
    "batch_size" : 8,
    "random_seed" : 7
}

In [26]:
train_dataloader = DataLoader(
            train_ds,  
            sampler = RandomSampler(train_ds), 
            batch_size = config["batch_size"]
        )

validation_dataloader = DataLoader(
            eval_ds, 
            sampler = RandomSampler(eval_ds), 
            batch_size = config["batch_size"]
        )

test_dataloader = DataLoader(
            test_ds, 
            sampler = RandomSampler(test_ds), 
            batch_size = config["batch_size"]
        )

In [27]:
model.to("cuda:0")

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [28]:
EPOCHS = 5
LR = 2e-5
EPS = 1e-8

optimizer = AdamW(model.parameters(), lr = LR, eps = EPS)
total_steps = len(train_dataloader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                           num_warmup_steps = 0, 
                                           num_training_steps = total_steps)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [30]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [31]:
random.seed(config["random_seed"])
np.random.seed(config["random_seed"])
torch.manual_seed(config["random_seed"])
torch.cuda.manual_seed_all(config["random_seed"])

In [32]:
for i in train_dataloader:
  print(f"{i[0]} \n\n {i[1]} \n\n {i[2]}")
  break

tensor([[   0, 6323, 3678,  ...,    1,    1,    1],
        [   0, 4993,   41,  ...,    1,    1,    1],
        [   0, 1594,    5,  ...,    1,    1,    1],
        ...,
        [   0, 1106,   38,  ...,    1,    1,    1],
        [   0, 4993,   41,  ...,    1,    1,    1],
        [   0, 1106,   38,  ...,    1,    1,    1]]) 

 tensor([[1.0000, 0.0185, 0.1609,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 0.4544, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [1.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 0.7251,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 0.8803, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64) 

 tensor([0, 1, 1, 1, 2, 1, 1, 2])


In [ ]:
training_stats = []
epochs = EPOCHS

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))


        b_input_ids = batch[0].to("cuda:0")
        b_input_mask = batch[1].to("cuda:0")
        b_labels = batch[2].to("cuda:0")

        model.zero_grad()        


        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        b_input_ids = batch[0].to("cuda:0")
        b_input_mask = batch[1].to("cuda:0")
        b_labels = batch[2].to("cuda:0")

        with torch.no_grad():        


            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...
  Batch    40  of    205.    Elapsed: 0:00:15.
  Batch    80  of    205.    Elapsed: 0:00:30.
  Batch   120  of    205.    Elapsed: 0:00:45.
  Batch   160  of    205.    Elapsed: 0:01:00.
  Batch   200  of    205.    Elapsed: 0:01:14.

  Average training loss: 0.85
  Training epcoh took: 0:01:16

Running Validation...
  Accuracy: 0.65
  Validation Loss: 0.85
  Validation took: 0:00:06

======== Epoch 2 / 5 ========
Training...
  Batch    40  of    205.    Elapsed: 0:00:15.
  Batch    80  of    205.    Elapsed: 0:00:29.
  Batch   120  of    205.    Elapsed: 0:00:44.
  Batch   160  of    205.    Elapsed: 0:00:58.
  Batch   200  of    205.    Elapsed: 0:01:13.

  Average training loss: 0.82
  Training epcoh took: 0:01:15

Running Validation...
  Accuracy: 0.66
  Validation Loss: 0.80
  Validation took: 0:00:06

======== Epoch 3 / 5 ========
Training...
  Batch    40  of    205.    Elapsed: 0:00:14.
  Batch    80  of    205.    Elapsed: 0:00:29.
 

In [ ]:
pd.set_option('precision', 2)
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')

df_stats

In [ ]:
# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4, 5, 6, 7])

plt.show()

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for i in tqdm(range(len(test_premise))):

    encoded_dict = roberta_tok.encode_plus(
                        test_premise[i],                      # Sentence to encode.
                        test_hypothesis[i],
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        truncation=True,
                        padding = "max_length",
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
test_labels = torch.tensor(test_labels)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, test_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in tqdm(prediction_dataloader):
  # Add batch to GPU
  batch = tuple(t.to("cuda:0") for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  pred_labels = np.argmax(logits, axis=1)
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.extend(pred_labels.tolist())
  true_labels.extend(label_ids.tolist())

print('DONE.')

In [ ]:
result_report= classification_report(true_labels, predictions, digits=3, output_dict=True)
result_report

In [ ]:
confusion_matrix(true_labels, predictions)

### Test Code

In [63]:
_tids = [    0,  1106,    38,   524,    10,   205,  1294,     6,    38,    40,
          120,    10,   205,  4978,  7586,   318,    38,   524,    10,  1099,
         1294,     6,    38,    40,   120,    10,  1099,  4978,  7586, 15482,
           38,   524,    10,   205,  1294,    50,    10,  1099,  1294,     4,
            2,     2, 32730,    38,    40,   120,    10,   205,  4978,     4,
            2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1]

In [64]:
_x = [roberta_tok.decode(_val) for _val in _tids]

In [ ]:
train_exp[0]

In [ ]:
for i in range(len(_tids)):
  print(f"{train_norm_exp_mask[0][i]} ==== {_tids[i]} === {_x[i]}")

In [ ]:
# train_encoded_input_attn_mask[0].tolist().count(0)

In [ ]:
# train_norm_exp_mask[0].tolist().count(0)

In [ ]:
# for i in range(len(train_encoded_input_attn_mask[0])):
#   if train_encoded_input_attn_mask[0][i] != train_norm_exp_mask[0][i]:
#     print(f"{i} === {train_encoded_input_attn_mask[0][i]} === {train_norm_exp_mask[0][i]}")